In [0]:
from pyspark.sql.functions import col, sum, round, count, countDistinct, when, lit, rank, expr
from pyspark.sql.window import Window

# 1. Initialize Schema
spark.sql("CREATE SCHEMA IF NOT EXISTS sales_project.gold")

# 2. Load Silver tables - Only select columns that DEFINITELY exist
df_items = spark.table("sales_project.silver.dim_order_items").filter(col("active_flag") == 'Y') \
    .select("order_id", "product_id", "price", "freight_value") # Removed customer_id from here

df_products = spark.table("sales_project.silver.fact_products").filter(col("active_flag") == 'Y') \
    .select("product_id", "product_category_name")

df_orders = spark.table("sales_project.silver.dim_orders").filter(col("active_flag") == 'Y') \
    .select("order_id", "customer_id", "order_status", "order_purchase_timestamp", "order_delivered_customer_date")

df_customers = spark.table("sales_project.silver.fact_customers").filter(col("active_flag") == 'Y') \
    .select("customer_id", "customer_city", "customer_state")

# --- TABLE 1: EXECUTIVE SALES PERFORMANCE ---
# FIX: Join items to orders FIRST (to get the customer_id), then join to customers
df_sales_joined = df_items.join(df_orders, "order_id") \
                          .join(df_customers, "customer_id")

gold_sales_performance = df_sales_joined.select(
    "order_id",
    "customer_id",
    "customer_city",
    "customer_state",
    "order_status",
    (col("price").cast("double") + col("freight_value").cast("double")).alias("total_order_value"),
    col("order_purchase_timestamp").cast("timestamp").alias("purchase_at"),
    col("order_delivered_customer_date").cast("timestamp").alias("delivered_at")
).withColumn("delivery_days", expr("datediff(delivered_at, purchase_at)"))

gold_sales_performance.write.mode("overwrite").saveAsTable("sales_project.gold.fact_sales_performance")

# --- TABLE 2: PRODUCT SENTIMENT ---
gold_product_metrics = df_products.join(df_items, "product_id", "left") \
    .groupBy("product_id", "product_category_name") \
    .agg(
        count("order_id").alias("total_orders"),
        round(sum(col("price").cast("double")), 2).alias("total_revenue")
    ).withColumn(
        "inventory_status",
        when(col("total_orders") == 0, lit("Neglected/Low Demand"))
        .when(col("total_orders") > 50, lit("Best Seller"))
        .otherwise(lit("Active"))
    )

gold_product_metrics.write.mode("overwrite").saveAsTable("sales_project.gold.dim_product_metrics")

# --- TABLE 3: CUSTOMER 360 & LOYALTY ---
# FIX: Since order_items doesn't have customer_id, we must join it with orders 
# to find which customer bought which product.
df_cust_prod_map = df_items.join(df_orders.select("order_id", "customer_id"), "order_id")

fav_prod_window = Window.partitionBy("customer_id").orderBy(col("purchase_count").desc())

df_fav_product = df_cust_prod_map.groupBy("customer_id", "product_id") \
    .agg(count("order_id").alias("purchase_count")) \
    .withColumn("rank", rank().over(fav_prod_window)) \
    .filter(col("rank") == 1) \
    .select("customer_id", col("product_id").alias("favorite_product_id"))

gold_customer_loyalty = gold_sales_performance.groupBy("customer_id", "customer_city", "customer_state") \
    .agg(
        countDistinct("order_id").alias("total_orders_placed"),
        round(sum("total_order_value"), 2).alias("lifetime_value")
    ).join(df_fav_product, "customer_id", "left")

gold_customer_loyalty.write.mode("overwrite").saveAsTable("sales_project.gold.dim_customer_loyalty")

print("Gold Layer Complete. Mapping logic adjusted for order_items schema.")

In [0]:
# Check the Sales Performance table
display(spark.table("sales_project.gold.fact_sales_performance").limit(10))

# Check the Customer Loyalty table
display(spark.table("sales_project.gold.dim_customer_loyalty").limit(10))

# Check the Product Metrics
display(spark.table("sales_project.gold.dim_product_metrics").limit(10))